# Lumber Parts
References:
- Table of standard lumber sizes  
  http://mistupid.com/homeimpr/lumber.htm
- TODO: https://www.engineeringtoolbox.com/green-kiln-dried-pressure-treated-lumber-weights-d_1860.html

In [134]:
# !conda install -y -n cq pandas requests beautifulsoup4

In [9]:
import pandas as pd
import requests
import bs4

In [13]:
lumber_sizes = "http://mistupid.com/homeimpr/lumber.htm"
resp = requests.get(lumber_sizes)
bs = bs4.BeautifulSoup(resp.text)

In [39]:
colnames = ['nominal', 'imperial', 'metric']

def extract_table(bs):
    tbl_html = bs.find_all('table')[2]
    # yield colnames
    for row in tbl_html.find_all('tr')[2:]:
        values = []
        for cell in row.find_all('td'):
            value = cell.text.replace(" ","").replace("\n","")
            values.append(value)
        yield values

rows = list(extract_table(bs))
rows

[['1"x2"', '3/4"x1-1/2"', '19x38mm'],
 ['1"x3"', '3/4"x2-1/2"', '19x64mm'],
 ['1"x4"', '3/4"x3-1/2"', '19x89mm'],
 ['1"x5"', '3/4"x4-1/2"', '19x114mm'],
 ['1"x6"', '3/4"x5-1/2"', '19x140mm'],
 ['1"x7"', '3/4"x6-1/4"', '19x159mm'],
 ['1"x8"', '3/4"x7-1/4"', '19x184mm'],
 ['1"x10"', '3/4"x9-1/4"', '19x235mm'],
 ['1"x12"', '3/4"x11-1/4"', '19x286mm'],
 ['1-1/4"x4"', '1"x3-1/2"', '25x89mm'],
 ['1-1/4"x6"', '1"x5-1/2"', '25x140mm'],
 ['1-1/4"x8"', '1"x7-1/4"', '25x184mm'],
 ['1-1/4"x10"', '1"x9-1/4"', '25x235mm'],
 ['1-1/4"x12"', '1"x11-1/4"', '25x286mm'],
 ['1-1/2"x4"', '1-1/4"x3-1/2"', '32x89mm'],
 ['1-1/2"x6"', '1-1/4"x5-1/2"', '32x140mm'],
 ['1-1/2"x8"', '1-1/4"x7-1/4"', '32x184mm'],
 ['1-1/2"x10"', '1-1/4"x9-1/4"', '32x235mm'],
 ['1-1/2"x12"', '1-1/4"x11-1/4"', '32x286mm'],
 ['2"x2"', '1-1/2"x1-1/2"', '38x38mm'],
 ['2"x4"', '1-1/2"x3-1/2"', '38x89mm'],
 ['2"x6"', '1-1/2"x5-1/2"', '38x140mm'],
 ['2"x8"', '1-1/2"x7-1/4"', '38x184mm'],
 ['2"x10"', '1-1/2"x9-1/4"', '38x235mm'],
 ['2"x12"',

In [62]:
#from decimal import Decimal
from fractions import Fraction

def nominal2fraction(str_):
    _str = str_.rstrip('"')
    components = _str.split("-")
    value = Fraction(components.pop(0))
    if components:
        value += Fraction(components.pop())
    return value

def test_nominal2fraction():
    ios = [
        ["3/4\"", Fraction("3/4")],
        ["1-1/2", Fraction("3/2")],
        ["3-1/2", Fraction("7/2")],
    ]
    for io in ios:
        output = nominal2fraction(io[0])
        print(output, io[1])
        assert output == io[1]
    
test_nominal2fraction()

3/4 3/4
3/2 3/2
7/2 7/2


In [89]:
def generate_object(row):
    nominal, imperial, metric = row
    obj = {
        "_row": row,
        "nominal": nominal,
        "imperial": imperial,
        "metric": metric
    }
    obj["nominal_h"], obj["nominal_w"] = nominal.split("x")
    obj["imperial_h"], obj["imperial_w"] = imperial.split("x")
    obj["metric_h"], obj["metric_w"] = metric.rstrip("m").split("x")
    
    obj["nominal_h_fraction"] = nominal2fraction(obj["nominal_h"])
    obj["nominal_w_fraction"] = nominal2fraction(obj["nominal_w"])
    obj["nominal_h_float"] = float(obj["nominal_h_fraction"])
    obj["nominal_w_float"] = float(obj["nominal_w_fraction"])
    
    
    obj["imperial_h_fraction"] = nominal2fraction(obj["imperial_h"])
    obj["imperial_w_fraction"] = nominal2fraction(obj["imperial_w"])
    obj["imperial_h_float"] = float(obj["imperial_h_fraction"])
    obj["imperial_w_float"] = float(obj["imperial_w_fraction"])
    
    obj["metric_h_float"] = float(obj["metric_h"])
    obj["metric_w_float"] = float(obj["metric_w"])
    
    obj['clsstr'] = nominal2clsstr(obj["nominal"])

    return obj

def nominal2clsstr(str_):
    return str_.replace('"','').replace("-","__").replace("/","_")

def test_nominal2clsstr():
    ios = [
        ["1x4", "1x4"],
        ['1-1/4\"x3-3/4"', "1__1_4x3__3_4"]
    ]
    for io in ios:
        assert nominal2clsstr(io[0]), io[1]

test_nominal2clsstr()
        
import json

class DecimalJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Fraction):
            return repr(obj)
        return json.JSONEncoder.default(self, obj)

import pprint
def test_generate_object(rows):
    for row in rows:
        obj = generate_object(row)
        #print(pprint.pformat(obj, indent=2))
        #print(obj)
        print(json.dumps(obj, cls=DecimalJSONEncoder, indent=1, sort_keys=True))
    #assert False
    
test_generate_object(rows)

{
 "_row": [
  "1\"x2\"",
  "3/4\"x1-1/2\"",
  "19x38mm"
 ],
 "clsstr": "1x2",
 "imperial": "3/4\"x1-1/2\"",
 "imperial_h": "3/4\"",
 "imperial_h_float": 0.75,
 "imperial_h_fraction": "Fraction(3, 4)",
 "imperial_w": "1-1/2\"",
 "imperial_w_float": 1.5,
 "imperial_w_fraction": "Fraction(3, 2)",
 "metric": "19x38mm",
 "metric_h": "19",
 "metric_h_float": 19.0,
 "metric_w": "38",
 "metric_w_float": 38.0,
 "nominal": "1\"x2\"",
 "nominal_h": "1\"",
 "nominal_h_float": 1.0,
 "nominal_h_fraction": "Fraction(1, 1)",
 "nominal_w": "2\"",
 "nominal_w_float": 2.0,
 "nominal_w_fraction": "Fraction(2, 1)"
}
{
 "_row": [
  "1\"x3\"",
  "3/4\"x2-1/2\"",
  "19x64mm"
 ],
 "clsstr": "1x3",
 "imperial": "3/4\"x2-1/2\"",
 "imperial_h": "3/4\"",
 "imperial_h_float": 0.75,
 "imperial_h_fraction": "Fraction(3, 4)",
 "imperial_w": "2-1/2\"",
 "imperial_w_float": 2.5,
 "imperial_w_fraction": "Fraction(5, 2)",
 "metric": "19x64mm",
 "metric_h": "19",
 "metric_h_float": 19.0,
 "metric_w": "64",
 "metric_w_floa

In [133]:
UNIT_TEXT = {
    "metric": "metric (mm)",
    "imperial": "imperial (in)"
}

def generate_class(obj, unit='imperial'):
    if unit not in ["metric", "imperial"]:
        raise ValueError("unit must be either 'metric' or 'imperial'")
    class_template = '''
@register("{dict_entry}")
class Lumber{clsstr}(Lumber):
    """a {nominal} ({actual_size}) piece of Lumber"""
    height = PositiveFloat({height}, doc="height of lumber in {unit_text} units (default: {height})")
    width = PositiveFloat({width}, doc="thickness of lumber in {unit_text} units (default: {width})")
'''
    
    context = {}
    context['unit_text'] = UNIT_TEXT.get(unit)
    context['height'] = obj["%s_h_float" % unit]
    context['width'] = obj["%s_w_float" % unit]
    context['actual_size'] = obj[unit]
    context['dict_entry'] = obj["nominal"].replace('"','')
    return class_template.format(**context, **obj)
    
  
DEFAULT_LENGTH = {
    "metric": 304.8,  # ~= 12.0in
    "imperial": 12.0
}
    
def generate_classes(objects, unit='imperial', template='wood_dark'):
    length = DEFAULT_LENGTH.get(unit)
    if length is None:
        raise ValueError("units must be either 'metric' (mm) or 'imperial'")
    unit_text = UNIT_TEXT.get(unit)
    lumbercls = '''
"""
Lumber classes for {unit_text} units
"""
import cqparts
from cqparts.params import PositiveFloat
from cqparts.display import render_props

class Lumber(cqparts.Part):
    length = PositiveFloat({length}, doc="length of lumber in {unit_text} units (default: {length})")
    _render = render_props(template='{template}')

    def make(self):
        return (
            cq.Workplane("XY")
            .box(self.length, self.height, self.width))


CLASSES = {{}}

def register(cls, *keys):
    for key in keys:
        CLASSES[key] = cls
    return cls
'''
    output = []
    output.append(lumbercls.format(length=length, unit_text=unit_text, template=template))
    for obj in objects:
        output.append(generate_class(obj, unit=unit))
    return '\n'.join(output)

def test_generate_classes():
    objs = [generate_object(row) for row in rows]
    for unit in ["imperial", "metric"]:
        print("\n####  %s\n" % unit)
        output = generate_classes(objs, unit=unit)
        print(output)

test_generate_classes()


####  imperial


"""
Lumber classes for imperial (in) units
"""
import cqparts
from cqparts.params import PositiveFloat
from cqparts.display import render_props

class Lumber(cqparts.Part):
    length = PositiveFloat(12.0, doc="length of lumber in imperial (in) units (default: 12.0)")
    _render = render_props(template='wood_dark')

    def make(self):
        return (
            cq.Workplane("XY")
            .box(self.length, self.height, self.width))


CLASSES = {}

def register(cls, *keys):
    for key in keys:
        CLASSES[key] = cls
    return cls


@register("1x2")
class Lumber1x2(Lumber):
    """a 1"x2" (3/4"x1-1/2") piece of Lumber"""
    height = PositiveFloat(0.75, doc="height of lumber in imperial (in) units (default: 0.75)")
    width = PositiveFloat(1.5, doc="thickness of lumber in imperial (in) units (default: 1.5)")


@register("1x3")
class Lumber1x3(Lumber):
    """a 1"x3" (3/4"x2-1/2") piece of Lumber"""
    height = PositiveFloat(0.75, doc="height of lumber in 